<a href="https://colab.research.google.com/github/EmmaHsueh/PL_project/blob/main/FinanceTracker_and_ManagementTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Personal Finance Tracker and Budget Management Tool**

[Google Sheet Link](https://docs.google.com/spreadsheets/d/1UzNgDMKD_WH3uhfQXdrBPM_z8oFHLh77yahMbu5KlHo/edit?gid=0#gid=0)

In [ ]:
# ==================== 安裝套件 ====================
!pip install gspread oauth2client gradio plotly seaborn pandas

In [ ]:

# ==================== 匯入套件 ====================
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.colab import auth
import gspread
from google.auth import default
import pandas as pd
import json
import csv
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import gradio as gr
from io import StringIO

In [ ]:
# ==================== Google Sheets 認證 ====================
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 開啟你的 Google Sheet
SHEET_URL = "https://docs.google.com/spreadsheets/d/1UzNgDMKD_WH3uhfQXdrBPM_z8oFHLh77yahMbu5KlHo/edit?usp=sharing"
sheet = gc.open_by_url(SHEET_URL).sheet1


In [ ]:
# ==================== 資料結構 ====================
transactions = []  # 儲存所有交易記錄
budgets = {}  # 儲存預算設定

# 預設類別
CATEGORIES = ["飲食", "交通", "娛樂", "購物", "醫療", "教育", "其他", "薪資", "投資", "獎金"]

In [ ]:
# ==================== 核心功能函數 ====================

def load_from_sheet():
    """從 Google Sheets 載入資料"""
    global transactions
    try:
        data = sheet.get_all_records()
        transactions = [dict(row) for row in data]
        return f"✅ 成功載入 {len(transactions)} 筆記錄"
    except Exception as e:
        return f"❌ 載入失敗: {str(e)}"

def save_to_sheet():
    """儲存資料到 Google Sheets"""
    try:
        # 清空工作表
        sheet.clear()

        # 寫入標題
        headers = ["日期", "類別", "類型", "金額", "備註", "ID"]
        sheet.append_row(headers)

        # 寫入資料
        for t in transactions:
            row = [
                t.get('日期', ''),
                t.get('類別', ''),
                t.get('類型', ''),
                t.get('金額', 0),
                t.get('備註', ''),
                t.get('ID', '')
            ]
            sheet.append_row(row)

        return f"✅ 成功儲存 {len(transactions)} 筆記錄到 Google Sheets"
    except Exception as e:
        return f"❌ 儲存失敗: {str(e)}"

def add_transaction(date, category, trans_type, amount, note):
    """新增交易記錄"""
    try:
        transaction = {
            "ID": f"T{len(transactions)+1:04d}",
            "日期": date,
            "類別": category,
            "類型": trans_type,
            "金額": float(amount),
            "備註": note
        }
        transactions.append(transaction)
        save_to_sheet()
        return f"✅ 成功新增: {trans_type} ${amount} ({category})"
    except Exception as e:
        return f"❌ 新增失敗: {str(e)}"

def delete_transaction(trans_id):
    """刪除交易記錄"""
    global transactions
    original_len = len(transactions)
    transactions = [t for t in transactions if t.get('ID') != trans_id]

    if len(transactions) < original_len:
        save_to_sheet()
        return f"✅ 成功刪除交易 {trans_id}"
    else:
        return f"❌ 找不到交易 {trans_id}"

def query_transactions(start_date, end_date, category, trans_type):
    """查詢交易記錄"""
    filtered = transactions.copy()

    if start_date:
        filtered = [t for t in filtered if t['日期'] >= start_date]
    if end_date:
        filtered = [t for t in filtered if t['日期'] <= end_date]
    if category != "全部":
        filtered = [t for t in filtered if t['類別'] == category]
    if trans_type != "全部":
        filtered = [t for t in filtered if t['類型'] == trans_type]

    if not filtered:
        return "查無資料"

    df = pd.DataFrame(filtered)
    return df.to_string(index=False)

In [ ]:

# ==================== 匯出/匯入功能 ====================

def export_csv():
    """匯出 CSV"""
    df = pd.DataFrame(transactions)
    csv_path = "/content/transactions.csv"
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    return csv_path

def export_json():
    """匯出 JSON"""
    json_path = "/content/transactions.json"
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(transactions, f, ensure_ascii=False, indent=2)
    return json_path

def import_csv(file):
    """匯入 CSV"""
    global transactions
    try:
        df = pd.read_csv(file.name)
        transactions = df.to_dict('records')
        save_to_sheet()
        return f"✅ 成功匯入 {len(transactions)} 筆記錄"
    except Exception as e:
        return f"❌ 匯入失敗: {str(e)}"

def import_json(file):
    """匯入 JSON"""
    global transactions
    try:
        with open(file.name, 'r', encoding='utf-8') as f:
            transactions = json.load(f)
        save_to_sheet()
        return f"✅ 成功匯入 {len(transactions)} 筆記錄"
    except Exception as e:
        return f"❌ 匯入失敗: {str(e)}"


In [ ]:
# ==================== 視覺化功能 ====================

def plot_trend():
    """收支趨勢圖"""
    if not transactions:
        return None

    df = pd.DataFrame(transactions)
    df['日期'] = pd.to_datetime(df['日期'])
    df['金額_signed'] = df.apply(lambda x: x['金額'] if x['類型']=='收入' else -x['金額'], axis=1)

    daily = df.groupby('日期')['金額_signed'].sum().reset_index()

    fig = px.line(daily, x='日期', y='金額_signed',
                  title='每日收支趨勢',
                  labels={'金額_signed': '金額 (收入為正/支出為負)'})
    fig.update_traces(line_color='#3b82f6', line_width=3)
    return fig

def plot_category_pie():
    """支出類別圓餅圖"""
    if not transactions:
        return None

    df = pd.DataFrame(transactions)
    expenses = df[df['類型'] == '支出']

    if expenses.empty:
        return None

    category_sum = expenses.groupby('類別')['金額'].sum()

    fig = px.pie(values=category_sum.values, names=category_sum.index,
                 title='支出類別分布',
                 color_discrete_sequence=px.colors.qualitative.Set3)
    return fig

def plot_income_expense():
    """收入vs支出對比"""
    if not transactions:
        return None

    df = pd.DataFrame(transactions)
    summary = df.groupby('類型')['金額'].sum()

    fig = go.Figure(data=[
        go.Bar(x=summary.index, y=summary.values,
               marker_color=['#10b981', '#ef4444'])
    ])
    fig.update_layout(title='收入 vs 支出總額',
                      xaxis_title='類型',
                      yaxis_title='金額')
    return fig

def plot_monthly_comparison():
    """月度收支對比"""
    if not transactions:
        return None

    df = pd.DataFrame(transactions)
    df['日期'] = pd.to_datetime(df['日期'])
    df['月份'] = df['日期'].dt.to_period('M').astype(str)

    monthly = df.pivot_table(values='金額',
                             index='月份',
                             columns='類型',
                             aggfunc='sum',
                             fill_value=0)

    fig = go.Figure()
    for col in monthly.columns:
        fig.add_trace(go.Bar(name=col, x=monthly.index, y=monthly[col]))

    fig.update_layout(title='月度收支對比',
                      barmode='group',
                      xaxis_title='月份',
                      yaxis_title='金額')
    return fig

In [ ]:
# ==================== Gradio 介面 ====================

def create_ui():
    with gr.Blocks(title="💰 個人財務管理系統", theme=gr.themes.Soft()) as demo:
        gr.Markdown("# 💰 個人財務追蹤與預算管理系統")
        gr.Markdown("### 📊 全方位管理你的收入與支出")

        with gr.Tabs():
            # Tab 1: 新增記錄
            with gr.Tab("➕ 新增記錄"):
                with gr.Row():
                    date_input = gr.Textbox(label="日期", value=datetime.now().strftime("%Y-%m-%d"))
                    category_input = gr.Dropdown(choices=CATEGORIES, label="類別")
                with gr.Row():
                    type_input = gr.Radio(choices=["收入", "支出"], label="類型", value="支出")
                    amount_input = gr.Number(label="金額")
                note_input = gr.Textbox(label="備註")
                add_btn = gr.Button("新增交易", variant="primary")
                add_output = gr.Textbox(label="結果")

                add_btn.click(add_transaction,
                             inputs=[date_input, category_input, type_input, amount_input, note_input],
                             outputs=add_output)

            # Tab 2: 查詢記錄
            with gr.Tab("🔍 查詢記錄"):
                with gr.Row():
                    query_start = gr.Textbox(label="開始日期 (YYYY-MM-DD)", value="2024-01-01")
                    query_end = gr.Textbox(label="結束日期 (YYYY-MM-DD)", value="2025-12-31")
                with gr.Row():
                    query_category = gr.Dropdown(choices=["全部"] + CATEGORIES, label="類別", value="全部")
                    query_type = gr.Dropdown(choices=["全部", "收入", "支出"], label="類型", value="全部")
                query_btn = gr.Button("查詢")
                query_output = gr.Textbox(label="查詢結果", lines=10)

                query_btn.click(query_transactions,
                               inputs=[query_start, query_end, query_category, query_type],
                               outputs=query_output)

            # Tab 3: 刪除記錄
            with gr.Tab("🗑️ 刪除記錄"):
                delete_id = gr.Textbox(label="輸入要刪除的交易ID (例如: T0001)")
                delete_btn = gr.Button("刪除", variant="stop")
                delete_output = gr.Textbox(label="結果")

                delete_btn.click(delete_transaction, inputs=delete_id, outputs=delete_output)

            # Tab 4: 視覺化分析
            with gr.Tab("📊 視覺化分析"):
                gr.Markdown("### 📈 資料視覺化")
                with gr.Row():
                    refresh_btn = gr.Button("🔄 更新圖表", variant="primary")

                with gr.Row():
                    trend_plot = gr.Plot(label="收支趨勢")
                    pie_plot = gr.Plot(label="支出類別分布")

                with gr.Row():
                    bar_plot = gr.Plot(label="收入vs支出")
                    monthly_plot = gr.Plot(label="月度對比")

                refresh_btn.click(
                    lambda: (plot_trend(), plot_category_pie(), plot_income_expense(), plot_monthly_comparison()),
                    outputs=[trend_plot, pie_plot, bar_plot, monthly_plot]
                )

            # Tab 5: 資料管理
            with gr.Tab("💾 資料管理"):
                gr.Markdown("### 📥 匯入/匯出資料")

                with gr.Row():
                    load_btn = gr.Button("從 Google Sheets 載入")
                    save_btn = gr.Button("儲存到 Google Sheets")
                io_output = gr.Textbox(label="結果")

                with gr.Row():
                    csv_btn = gr.Button("匯出 CSV")
                    json_btn = gr.Button("匯出 JSON")

                csv_file = gr.File(label="CSV 檔案")
                json_file = gr.File(label="JSON 檔案")

                with gr.Row():
                    import_csv_file = gr.File(label="上傳 CSV")
                    import_json_file = gr.File(label="上傳 JSON")
                import_output = gr.Textbox(label="匯入結果")

                load_btn.click(load_from_sheet, outputs=io_output)
                save_btn.click(save_to_sheet, outputs=io_output)
                csv_btn.click(export_csv, outputs=csv_file)
                json_btn.click(export_json, outputs=json_file)
                import_csv_file.change(import_csv, inputs=import_csv_file, outputs=import_output)
                import_json_file.change(import_json, inputs=import_json_file, outputs=import_output)

        gr.Markdown("---")
        gr.Markdown("💡 **使用提示**: 先點擊「從 Google Sheets 載入」來同步資料")

    return demo

In [ ]:

# ==================== 啟動應用 ====================
if __name__ == "__main__":
    # 初次載入資料
    load_from_sheet()

    # 啟動 Gradio
    app = create_ui()
    app.launch(share=True, debug=True)

"""

## 🎯 使用說明

### 1️⃣ 設定 Google Sheet
你的試算表需要有以下欄位:

日期 | 類別 | 類型 | 金額 | 備註 | ID
"""

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://39f7950c9a3c618c5a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://39f7950c9a3c618c5a.gradio.live


'\n\n## 🎯 使用說明\n\n### 1️⃣ 設定 Google Sheet\n你的試算表需要有以下欄位:\n\n日期 | 類別 | 類型 | 金額 | 備註 | ID\n'